In [1]:
import numpy as np
import SimpleITK as sitk
import matplotlib.pyplot as plt
from PIL import Image
from scipy import signal
import cv2

import os
import random

In [22]:
img = sitk.ReadImage('/home/peter/Documents/data_registered/05z1.nii')

In [16]:
path = '/home/peter/Documents/dose_estimator-git/data/data_corrected/'
path_out = '/home/peter/Documents/dose_estimator-git/data/data_3d'

In [18]:
# go through file system
pet = []
ct = []
dose= []

blacklist = ['06z1', '07z2', '12z4', '20z1', '23z1', '27z1', '62z1', '62z2']

for r, d, f in os.walk(path):
    for file in f:
        if '.nii' in file and file[:4] not in blacklist:
            if 'pet' in file:
                pet.append(os.path.join(r, file))
            elif 'ct' in file:
                ct.append(os.path.join(r, file))
            else:
                dose.append(os.path.join(r, file))
print(pet[0])

/home/peter/Documents/dose_estimator-git/data/data_corrected/05z4_pet.nii.gz


In [5]:
len(pet)

40

In [ ]:
# SAVE IN NEW FOLDER STRUCTURE ------------------------------------------------------------------------------------------

In [6]:
ref_size = (81, 128, 128)

In [21]:
# save pet images (third dimension must match)
wrong_imgs = []

for p in pet:
    # load image
    orig_nii = sitk.ReadImage(p)
    orig = sitk.GetArrayFromImage(orig_nii)
    #print(orig.shape)
    out = np.zeros(ref_size)
    
    # check size
    if orig.shape[0] <= 82:
        if orig.shape != ref_size: 
            for s in range(81): #orig.shape[0]
                out[s,:,:] = cv2.resize(orig[s,:,:], ref_size[1:])
        else:
            for s in range(81):
                out[s,:,:] = orig[s,:,:]
                
        # convert pack to NIFTI
        itkimage = sitk.GetImageFromArray(out, isVector=False)

        # copy metadata
        for k in orig_nii.GetMetaDataKeys():
            itkimage.SetMetaData(k, orig_nii.GetMetaData(k))
        itkimage.SetMetaData('dim[1]', str(ref_size[1]))
        itkimage.SetMetaData('dim[2]', str(ref_size[2]))
        itkimage.SetMetaData('dim[3]', str(ref_size[0]))

        # new filename
        filename = p.split('/')[-1].split('_')[0] + '_pet.nii.gz'

        # save to new folder
        sitk.WriteImage(itkimage, os.path.join(path_out, filename), True)
    else:
        print(p.split('/')[-1].split('_')[0])
        wrong_imgs.append(p.split('/')[-1].split('_')[0])

07z3
05z9
21z1


In [7]:
wrong_imgs = ['07z3', '05z9', '21z1']

In [23]:
# save ct images

for p in ct:
    # load image
    orig_nii = sitk.ReadImage(p)
    orig = sitk.GetArrayFromImage(orig_nii)
    #print(orig.shape)
    out = np.zeros(ref_size)
    
    # check size
    if orig.shape[0] <= 82:
        if orig.shape != ref_size: 
            for s in range(81): #orig.shape[0]
                out[s,:,:] = cv2.resize(orig[s,:,:], ref_size[1:])
        else:
            for s in range(81):
                out[s,:,:] = orig[s,:,:]
                
        # convert pack to NIFTI
        itkimage = sitk.GetImageFromArray(out, isVector=False)

        # copy metadata
        for k in orig_nii.GetMetaDataKeys():
            itkimage.SetMetaData(k, orig_nii.GetMetaData(k))
        itkimage.SetMetaData('dim[1]', str(ref_size[1]))
        itkimage.SetMetaData('dim[2]', str(ref_size[2]))
        itkimage.SetMetaData('dim[3]', str(ref_size[0]))

        # new filename
        filename = p.split('/')[-1].split('_')[0] + '_ct.nii.gz'

        # save to new folder
        sitk.WriteImage(itkimage, os.path.join(path_out, filename), True)
    else:
        print(p.split('/')[-1].split('_')[0])

21z1
07z3
05z9


In [24]:
# save dose images

for p in dose:
    # load image
    orig_nii = sitk.ReadImage(p)
    orig = sitk.GetArrayFromImage(orig_nii)
    #print(orig.shape)
    out = np.zeros(ref_size)
    
    # check size
    # check size
    if orig.shape[0] <= 82:
        if orig.shape != ref_size: 
            for s in range(81): #orig.shape[0]
                out[s,:,:] = cv2.resize(orig[s,:,:], ref_size[1:])
        else:
            for s in range(81):
                out[s,:,:] = orig[s,:,:]
                
        # convert pack to NIFTI
        itkimage = sitk.GetImageFromArray(out, isVector=False)

        # copy metadata
        for k in orig_nii.GetMetaDataKeys():
            itkimage.SetMetaData(k, orig_nii.GetMetaData(k))
        itkimage.SetMetaData('dim[1]', str(ref_size[1]))
        itkimage.SetMetaData('dim[2]', str(ref_size[2]))
        itkimage.SetMetaData('dim[3]', str(ref_size[0]))

        # new filename
        filename = p.split('/')[-1].split('.')[0] + '.nii.gz'

        # save to new folder
        sitk.WriteImage(itkimage, os.path.join(path_out, filename), True)
    else:
        print(p.split('/')[-1].split('.')[0])

07z3
05z9
21z1


In [ ]:
# SAVE AS NUMPY FILES ---------------------------------------------------------------------------------------------------

In [8]:
pet_filt = []
for x in pet:
    if x.split('/')[-1].split('_')[0] not in wrong_imgs:
        pet_filt.append(x)

In [9]:
len(pet_filt)

37

In [10]:
# select test set
testset_size = 5



dataset = [x.split('/')[-1].split('_')[0] for x in pet_filt]
testset = ['15z1', '09z1', '05z4', '05z1', '20z2']#random.choices(dataset,k=testset_size)
trainset = [x for x in dataset if x not in testset]
print(testset)

['15z1', '09z1', '05z4', '05z1', '20z2']


In [14]:
len(trainset)

32

In [20]:
pet_final = np.zeros((len(trainset)*81, 128, 128))
ct_final = np.zeros((len(trainset)*81, 128, 128))
dose_final = np.zeros((len(trainset)*81, 128, 128))

for idx, i in enumerate(trainset):
    print(i)
    pet_img = sitk.GetArrayFromImage(sitk.ReadImage(os.path.join(path, i + '_pet.nii.gz')))
    ct_img = sitk.GetArrayFromImage(sitk.ReadImage(os.path.join(path, i + '_ct.nii.gz')))
    dose_img = sitk.GetArrayFromImage(sitk.ReadImage(os.path.join(path, i + '.nii.gz')))
    print(pet_img.shape[0])
    for x in range(pet_img.shape[0]):
        pet_final[idx*81+x,:,:] = pet_img[x,:,:]
        ct_final[idx*81+x,:,:] = ct_img[x,:,:]
        dose_final[idx*81+x,:,:] = dose_img[x,:,:]
    del pet_img, ct_img, dose_img
    
np.save(f"{path_out}/pet_train.npy", pet_final)
np.save(f"{path_out}/ct_train.npy", ct_final)
np.save(f"{path_out}/dose_train.npy", dose_final)

25z1
81
13z2
81
29z1
81
05z3
81
28z1
81
05z2
81
10z1
81
24z2
81
14z1
81
14z2
81
17z1
81
15z2
81
07z1
81
18z2
81
24z1
81
07z4
81
10z2
81
05z5
81
14z3
81
12z2
81
11z4
81
13z1
81
12z1
81
11z2
81
18z3
81
18z1
81
11z1
81
11z3
81
12z3
81
08z1
81
16z1
81
14z4
81


In [21]:
del pet_final, ct_final, dose_final

In [23]:
pet_final2 = np.zeros((len(testset)*81, 128, 128))
ct_final2 = np.zeros((len(testset)*81, 128, 128))
dose_final2 = np.zeros((len(testset)*81, 128, 128))

for idx, i in enumerate(testset):
    print(i)
    pet_img = sitk.GetArrayFromImage(sitk.ReadImage(os.path.join(path, i + '_pet.nii.gz')))
    ct_img = sitk.GetArrayFromImage(sitk.ReadImage(os.path.join(path, i + '_ct.nii.gz')))
    dose_img = sitk.GetArrayFromImage(sitk.ReadImage(os.path.join(path, i + '.nii.gz')))
    for x in range(pet_img.shape[0]):
        pet_final2[idx*81+x,:,:] = pet_img[x,:,:]
        ct_final2[idx*81+x,:,:] = ct_img[x,:,:]
        dose_final2[idx*81+x,:,:] = dose_img[x,:,:]
    del pet_img, ct_img, dose_img
    
np.save(f"{path_out}/pet_test.npy", pet_final2)
np.save(f"{path_out}/ct_test.npy", ct_final2)
np.save(f"{path_out}/dose_test.npy", dose_final2)

15z1
09z1
05z4
05z1
20z2


In [24]:
# save image name lists to text files
with open(f"{path_out}/train.txt", 'w') as f:
    for item in trainset:
        f.write("%s\n" % item)

with open(f"{path_out}/test.txt", 'w') as f:
    for item in testset:
        f.write("%s\n" % item)

In [26]:
text_file = open(f"{path_out}/test.txt", "r", encoding='utf8')
list1 = text_file.read().splitlines()
list1

['15z1', '09z1', '05z4', '05z1', '20z2']